In [2]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnableParallel
from dotenv import load_dotenv
import os

load_dotenv()

repo_id1 = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm1 = HuggingFaceEndpoint(
    repo_id = repo_id1,
    task = "text-generation", 
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_ACCESS_TOKEN")
)
model1 = ChatHuggingFace(llm=llm1)


repo_id2 = "google/gemma-3-27b-it"
llm2 = HuggingFaceEndpoint(
    repo_id=repo_id2,
    task="text-generation",
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_ACCESS_TOKEN")
)
model2 = ChatHuggingFace(llm=llm2)

parser = StrOutputParser()

prompt1 = PromptTemplate(
    template = "Generate short and siple notes from the following text \n {text}",
    input_variables=["text"]
)

prompt2 = PromptTemplate(
    template = "Generate 5 short question answers from the following text \n {text}",
    input_variables=["text"]
)

prompt3 = PromptTemplate(
    template = "merge the provided notes and quiz into a following document \n notes -> {notes} and quiz -> {quiz}",
    input_variables=["notes", "quiz"]
)

parallel_chain = RunnableParallel({
    "notes" : prompt1 | model1 | parser,
    "quiz" : prompt2 | model2 | parser
})

merge_chain = prompt3 | model1 | parser

chain = parallel_chain | merge_chain

text = """
Support Vector Machines (SVM) are a type of supervised machine learning algorithm widely used for 
classification and, to a lesser extent, regression tasks. The core idea behind SVM is to find the 
optimal hyperplane that separates data points of different classes with the maximum possible margin.
The data points that lie closest to this hyperplane are called support vectors, and they play a
crucial role in defining the decision boundary. For datasets that are not linearly separable,SVM 
uses kernel functions to project the data into higher-dimensional spaces where a linear separation
is possible. Common kernels include the linear, polynomial, and radial basis function (RBF) kernels.
SVMs are known for their high accuracy, especially in high-dimensional spaces, but they can be
 computationally intensive and sensitive to the choice of kernel and parameters.
"""
result = chain.invoke({"text": text})

print(result)


c:\Users\dheka\OneDrive\Desktop\11.Langchain_from_beginning\myenv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dheka\.cache\huggingface\hub\models--google--gemma-3-27b-it. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


 Support Vector Machines (SVM) are a type of supervised machine learning algorithm that can be used for classification and regression tasks. The main goal of an SVM is to find the optimal hyperplane that separates data points of different classes with the maximum margin. The data points that are closest to the hyperplane are called support vectors, and they are crucial for defining the decision boundary.

Handling non-linearly separable datasets is achieved in SVM by using kernel functions to project the data into higher dimensional spaces, where linear separation is possible. Common kernel functions used in SVM include linear, polynomial, and radial basis function (RBF).

SVMs are known for their high accuracy, especially in high-dimensional spaces. However, they can be computationally intensive and may require careful tuning of kernel and parameter choices.

Here are five short question-answer pairs based on the provided text:

1. Q: What type of machine learning algorithm are SVMs?


In [3]:
chain.get_graph().print_ascii()

              +---------------------------+              
              | Parallel<notes,quiz>Input |              
              +---------------------------+              
                  ***               ***                  
               ***                     ***               
             **                           **             
+----------------+                    +----------------+ 
| PromptTemplate |                    | PromptTemplate | 
+----------------+                    +----------------+ 
          *                                   *          
          *                                   *          
          *                                   *          
+-----------------+                  +-----------------+ 
| ChatHuggingFace |                  | ChatHuggingFace | 
+-----------------+                  +-----------------+ 
          *                                   *          
          *                                   *          
          *   